In [25]:
import numpy as np
import pprint
import sys
import matplotlib.pyplot as plt
import argparse
from scipy.special import softmax
if "../" not in sys.path:
    sys.path.append("../")
from lib.envs.gridworld import GridworldEnv
import json
import pdb
from collections import defaultdict
from functools import partial

pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

env.nA = 4
env.nS = 22

In [39]:
import pandas as pd
scg_toy = r"C:\Users\Tongtong\Desktop\UCL CSML\aa_game theory\code_SCG\SPG-master\SPG-master\data\SCG_toy.xlsx"
data = pd.read_excel(scg_toy)
data.head()

,ID,State,"(a1, b1)","(a1, b2)","(a2,b1)","(a2,b2)",R1,R2,R3,R4,phi1,phi2,phi3,phi4
0,0,{B0;B0},1,2,3,4,"(-2,-2)","(-0.5,-2)","(-2,-0.5)","(-1,-1)",-2.5,-1.0,-1.0,0.0
1,1,{B1;B1},5,6,7,8,"(-1.6,-1.6)","(-0.8,-2)","(-2,-0.8)","(-2,-2)",0.0,-0.4,-0.4,-1.6
2,2,{B2;B1},13,14,15,16,"(-0.8,-0.9)","(-0.8,-2)","(-2,-0.9)","(-2,-2)",0.0,-1.1,-1.2,-2.3
3,3,{B1;B2},17,18,19,20,"(-0.9,-0.8)","(-2,-0.8)","(-0.9,-2)","(-2,-2)",0.0,-1.2,-1.1,-2.3
4,4,{B2;B2},9,10,11,12,"(-1.8,-1.8)","(-0.9,-2)","(-2,-0.9)","(-2,-2)",0.0,-0.2,-0.2,-1.3


In [40]:
phi = data.loc[:,["phi1","phi2","phi3","phi4"]].values.tolist()
transition = data.iloc[:,2:6].values.tolist()

In [41]:
phi

[[-2.5, -1.0, -1.0, 0.0],
 [0.0, -0.4, -0.4, -1.6],
 [0.0, -1.1, -1.2, -2.3],
 [0.0, -1.2, -1.1, -2.3],
 [0.0, -0.2, -0.2, -1.3],
 [0.0, -0.4, -0.4, -1.6],
 [0.0, -1.1, -1.2, -2.3],
 [0.0, -1.2, -1.1, -2.3],
 [0.0, -0.2, -0.2, -1.3],
 [0.0, -0.4, -0.4, -1.6],
 [0.0, -1.1, -1.2, -2.3],
 [0.0, -1.2, -1.1, -2.3],
 [0.0, -0.2, -0.2, -1.3],
 [0.0, -1.2, -1.2, -2.4],
 [0.0, -1.1, -1.2, -2.3],
 [0.0, -1.2, -1.1, -2.3],
 [0.0, -1.1, -1.1, -2.2],
 [0.0, -1.2, -1.2, -2.4],
 [0.0, -1.1, -1.2, -2.3],
 [0.0, -1.1, -1.2, -2.3],
 [0.0, -1.1, -1.1, -2.2],
 [0.0, 0.0, 0.0, 0.0]]

In [42]:
x = np.array(phi) + 1
x[1, :] = x[1, :] + 10
x[2, :] = x[2, :] + 5
x[1, :] = x[1, :] + 1
x[4, :] = x[4, :] - 2
phi = x.tolist()
phi

[[-1.5, 0.0, 0.0, 1.0],
 [12.0, 11.6, 11.6, 10.4],
 [6.0, 4.9, 4.8, 3.7],
 [1.0, -0.19999999999999996, -0.10000000000000009, -1.2999999999999998],
 [-1.0, -1.2, -1.2, -2.3],
 [1.0, 0.6, 0.6, -0.6000000000000001],
 [1.0, -0.10000000000000009, -0.19999999999999996, -1.2999999999999998],
 [1.0, -0.19999999999999996, -0.10000000000000009, -1.2999999999999998],
 [1.0, 0.8, 0.8, -0.30000000000000004],
 [1.0, 0.6, 0.6, -0.6000000000000001],
 [1.0, -0.10000000000000009, -0.19999999999999996, -1.2999999999999998],
 [1.0, -0.19999999999999996, -0.10000000000000009, -1.2999999999999998],
 [1.0, 0.8, 0.8, -0.30000000000000004],
 [1.0, -0.19999999999999996, -0.19999999999999996, -1.4],
 [1.0, -0.10000000000000009, -0.19999999999999996, -1.2999999999999998],
 [1.0, -0.19999999999999996, -0.10000000000000009, -1.2999999999999998],
 [1.0, -0.10000000000000009, -0.10000000000000009, -1.2000000000000002],
 [1.0, -0.19999999999999996, -0.19999999999999996, -1.4],
 [1.0, -0.10000000000000009, -0.199999999

In [24]:
transition

[[1, 2, 3, 4],
 [5, 6, 7, 8],
 [13, 14, 15, 16],
 [17, 18, 19, 20],
 [9, 10, 11, 12],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21],
 [21, 21, 21, 21]]